In [1]:
# !pip install smolagents -U

In [2]:
# from huggingface_hub import notebook_login

# notebook_login()

In [3]:
# !pip install duckduckgo-search

In [4]:
# from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel

# agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=InferenceClientModel())

# agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

In [5]:
!pip install smolagents -U
!pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [7]:
from huggingface_hub import login

login()

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [10]:
def ask_mistral(messages):
    """
    Executes a chat conversation with Mistral using a prompt template.
    """
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(input_ids, max_new_tokens=256)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove user prompt to keep only the assistant's reply
    prompt = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return response.replace(prompt, "").strip()

In [12]:
def suggest_menu(occasion: str) -> str:
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "Three-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

def menu_agent(user_input: str) -> str:
    lower = user_input.lower()
    if "casual" in lower:
        return suggest_menu("casual")
    elif "formal" in lower:
        return suggest_menu("formal")
    elif "superhero" in lower:
        return suggest_menu("superhero")
    else:
        return suggest_menu("custom")

# Example call
print("Menu Suggestion:\n" + menu_agent("Prepare a formal menu for the party."))


Menu Suggestion:
Three-course dinner with wine and dessert.


In [13]:
import datetime

def planner_agent(tasks=None):
    if tasks is None:
        tasks = [
            ("Prepare the drinks", 30),
            ("Decorate the mansion", 60),
            ("Set up the menu", 45),
            ("Prepare the music and playlist", 45)
        ]

    now = datetime.datetime.now()
    total_minutes = sum(task[1] for task in tasks)
    ready_time = now + datetime.timedelta(minutes=total_minutes)

    print("Task Plan:")
    for task, minutes in tasks:
        print(f"  - {task} ({minutes} min)")

    print(f"\nStart time: {now.strftime('%H:%M')}")
    print(f"Party will be ready by: {ready_time.strftime('%H:%M')}")

# Example call
planner_agent()


Task Plan:
  - Prepare the drinks (30 min)
  - Decorate the mansion (60 min)
  - Set up the menu (45 min)
  - Prepare the music and playlist (45 min)

Start time: 10:44
Party will be ready by: 13:44


In [16]:
def ask_mistral(messages):
    """
    Executes a chat conversation with Mistral using a prompt template.
    Returns the model-generated assistant response only.
    """
    prompt_input = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)


    outputs = model.generate(
        prompt_input,
        max_new_tokens=256,
        attention_mask=torch.ones_like(prompt_input),
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    prompt_text = tokenizer.decode(prompt_input[0], skip_special_tokens=True)
    return full_output.replace(prompt_text, "").strip()



In [17]:
!pip install duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 39.5 MB/s eta 0:00:00


In [18]:
from duckduckgo_search import DDGS

def duckduckgo_search_agent(query: str, max_results=5):
    try:
        with DDGS() as ddgs:
            results = ddgs.text(query)
            if not results:
                print(f"No results found for: {query}")
                return
            print(f"DuckDuckGo Search Results for: '{query}'\n")
            for i, r in enumerate(results[:max_results], start=1):
                print(f"{i}. {r['title']}\n   {r['href']}\n   {r['body']}\n")
    except Exception as e:
        print(f"Error performing search: {e}")

# Example call
duckduckgo_search_agent("Best music for a superhero party")


DuckDuckGo Search Results for: 'Best music for a superhero party'

1. 23 Best Songs about Heroes and Superheroes - Music Grotto
   https://www.musicgrotto.com/songs-about-heroes/
   Superheroes and heroes in music create some of the most iconic tracks. Check out this list of these top songs about heroes and superheroes!

2. superhero theme songs - YouTube
   https://www.youtube.com/watch?v=9ZuWt7ZEFMQ
   Including: Superman, Green Lantern, X-men, Batman & The Avengers

3. superhero party music - playlist by Danny Malave | Spotify
   https://open.spotify.com/playlist/3DyU3cM65SqrEFs9YYJ7H6
   superhero party music · Playlist · 67 songs · 28 likes

4. Superhero Music
   https://music.youtube.com/playlist?list=PLvU3t3QkFPlvDs8lzaWL-CZPL1xJcYpLC
   The Best Hero Theme Songs Right Here! Spider-Man, 007, Mission Impossible, Batman and more!

5. Superhero Party Music - Spotify Playlist | Partyrama Blog
   https://blog.partyrama.co.uk/superhero-party-music-spotify-playlist/
   Superhero Party 